# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,516.587663,-0.807543,1234,515.780120,517.395207,1031.560241,1034.790413
6,115.036932,-0.568073,1234,114.468859,115.605006,228.937718,231.210011
7,861.603470,-0.317990,1234,861.285480,861.921460,1722.570960,1723.842921
8,547.735388,0.968353,1234,548.703741,546.767034,1097.407482,1093.534068
9,958.395415,-0.527423,1234,957.867992,958.922838,1915.735983,1917.845675
10,969.505590,0.216601,1234,969.722190,969.288989,1939.444381,1938.577979
11,25.199222,-0.437503,1234,24.761719,25.636725,49.523438,51.273451
12,361.334877,0.791199,1234,362.126075,360.543678,724.252151,721.087357
13,887.918834,-0.724531,1234,887.194303,888.643364,1774.388606,1777.286728
14,667.796142,0.618918,1234,668.415059,667.177224,1336.830119,1334.354449


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-515.780120,517.395207
6,-114.468859,115.605006
7,-861.285480,861.921460
8,-548.703741,546.767034
9,-957.867992,958.922838
10,-969.722190,969.288989
11,-24.761719,25.636725
12,-362.126075,360.543678
13,-887.194303,888.643364
14,-668.415059,667.177224


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-515.780120,517.395207
6,-114.468859,115.605006
7,-861.285480,861.921460
8,-548.703741,546.767034
9,-957.867992,958.922838
10,-969.722190,969.288989
11,-24.761719,25.636725
12,-362.126075,360.543678
13,-887.194303,888.643364
14,-668.415059,667.177224


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-515.78012047,  517.3952065 ],
       [-114.46885889,  115.6050057 ],
       [-861.28548011,  861.92146039],
       [-548.70374104,  546.76703413],
       [-957.86799165,  958.9228376 ],
       [-969.72219047,  969.28898947],
       [ -24.76171876,   25.63672549],
       [-362.1260754 ,  360.54367835],
       [-887.19430304,  888.64336417],
       [-668.41505943,  667.17722426]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1029.560241,-1034.790413
6,226.937718,-231.210011
7,1720.570960,-1723.842921
8,1095.407482,-1093.534068
9,1913.735983,-1917.845675
10,1937.444381,-1938.577979
11,47.523438,-51.273451
12,722.252151,-721.087357
13,1772.388606,-1777.286728
14,1334.830119,-1334.354449


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
